# Azure SQL Database
In this notebook we will access Azure SQL database

In [1]:
# install all the azure identity modules
%pip install azure.identity
%pip install pandas
# install all the azure sql database modules
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyodbc in c:\leetcode\python\.venv\lib\site-packages (5.1.0)




[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
import struct
import pyodbc
import pandas as pd 

class DatabaseConnection():
    def __init__(self, configuration : dict):
        server = configuration.get('server')
        if (not str(server).endswith('.database.windows.net')) :
            server = server + '.database.windows.net';
        self.sever = server;
        self.database = configuration.get('database');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        databaseToken = credential.get_token('https://database.windows.net/');
        print(databaseToken);
        tokenb = bytes(databaseToken[0], "UTF-8");
        exptoken = b'';
        for i in tokenb:
            exptoken += bytes({i});
            exptoken += bytes(1);
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
        connString = "Driver={ODBC Driver 17 for SQL Server};SERVER="+self.sever+";DATABASE="+self.database+"";
        SQL_COPT_SS_ACCESS_TOKEN = 1256; 
        self.connection = pyodbc.connect(connString, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});


    # write a database to database, mode = append or replace
    def insert(self, df : pd.DataFrame, table :str):
        """
        Execute a SQL write.
        @param df: Data Frame.        
        @param table: Table Name.
        """
        statement = '''
        INSERT INTO {table}
        (
            {columns}
        )
        VALUES({place_holders})
        '''
        # add protection on column name
        columns = df.columns.tolist();
        for i in range(len(columns)):
            columns[i] = '[' + columns[i] + ']';
        statement = statement.format(table = table, 
                                     columns = (',').join(columns), 
                                     place_holders = (',').join(['?'] * len(df.columns)));
        print(statement);
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.fast_executemany = True;
        cursor.executemany(statement, df.values.tolist());
        cursor.commit();
        cursor.close();

    # execute sql command
    def execute(self, statement : str):
        """
        Execute a SQL command.
        @param query: query.        
        """
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.execute(statement);
        cursor.commit();
        cursor.close();


    def query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        # Create a Pandas dataframe from the results
        
        df = pd.read_sql(query, self.connection);

        # Print the dataframe
        print(df);
        return df

    def close(self):
        """
        Close the Database client.
        """
        self.connection.close();


databaseConn = DatabaseConnection({'server' : 'eppmreportingdev.database.windows.net', 'database' :'DCEPDev'});
df = databaseConn.query('SELECT * FROM [CSPW].[GPURetrofit_Signal_Fact]');
databaseConn.close();
print(df);
    


AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCIsImtpZCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0LyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0Ny8iLCJpYXQiOjE3MTUxMTE1MDEsIm5iZiI6MTcxNTExMTUwMSwiZXhwIjoxNzE1MTE2NDAyLCJfY2xhaW1fbmFtZXMiOnsiZ3JvdXBzIjoic3JjMSJ9LCJfY2xhaW1fc291cmNlcyI6eyJzcmMxIjp7ImVuZHBvaW50IjoiaHR0cHM6Ly9ncmFwaC53aW5kb3dzLm5ldC83MmY5ODhiZi04NmYxLTQxYWYtOTFhYi0yZDdjZDAxMWRiNDcvdXNlcnMvOTJlYzgxNWYtZGM0Yy00ZWZmLWFmNGYtMzRjMjc5ZGI4YmIzL2dldE1lbWJlck9iamVjdHMifX0sImFjciI6IjEiLCJhaW8iOiJBZVFBRy84V0FBQUFORVovVXhqQXVUUllLSllObXJMMWFwa2VZSit4QU1yMmUvRkVzWjl5NWJIWXdhRFFCY0Q3WE9La3puako3OFkxRlZ2Y0NPSkFmQmtYbE5lbjAyTXVrUjFzdHhBa0RyWFd2VWhSSmhMaHdtY1Q1Vy9wUVloS1lnS3ZPdlp5d2RlQ21kRDZhVWxaK2FHYlM4bFhYUVlLNXh4eVZGOGd1RU0zRUk1dU9WbnM5eVpIMmV3VmhKM2d6dUpmN0pUb3hmR3VLY0x2TnZkOFRpMHVYMjJvaW5rK2M5TWFhYVB4aTV3YlFUVzkyZmk1ZHBTN0JrMnYvcVFwckhrZGxY

C:\Users\huanminw\AppData\Local\Temp\ipykernel_1032\1079825882.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection);


      PORVersion PublishedDate  OrderType Region    Metro     AzureRegion  \
0       20231219    2023-11-16  Execution   AMER  Phoenix       West US 3   
1       20231219    2023-11-16  Execution   APAC   Sydney  Australia East   
2       20231219    2023-11-15  Execution   AMER  Boydton       East US 2   
3       20231219    2023-11-13  Execution   AMER  Boydton       East US 2   
4       20231219    2023-11-13  Execution   AMER  Boydton       East US 2   
...          ...           ...        ...    ...      ...             ...   
1461    20231219    2023-02-21  Execution   APAC   Sydney  Australia East   
1462    20231219    2023-02-21  Execution   APAC   Sydney  Australia East   
1463    20231219    2023-02-21  Execution   EMEA   Warsaw  Poland Central   
1464    20231219    2023-02-21  Execution   EMEA   Warsaw  Poland Central   
1465    20231219    2023-02-21  Execution   EMEA   Warsaw  Poland Central   

        AZ     DC    COLO  Type      Racks  GPU MWs Build/Lease  Colo MWs  